In [159]:
import sqlite3
import collections
import json

data = json.load(open("mana.json"))

In [160]:
def generate_mana_id(month, day, title, verse):
    return str(month) + "-" + str(day) + "_" + str(title).strip().lower().replace(" ", "") + "_" + str(verse).strip().replace(" ", "").lower()

class Mana:
    def __init__(self, columns):
        for col in columns:
            self.__setattr__(col, "")

    def insert_into_db(self, database_path):
        con = sqlite3.connect(database_path)
        cur = con.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS mana (
            id varchar(100) PRIMARY KEY,
            month int NOT NULL,
            vakiteny varchar(50),
            lohateny varchar(50),
            content text,
            F varchar(10));""")

        query = "insert into mana "
        map_ = {"columns":"(", "values":" values ("}

        it = 0
        for (attr_name, value) in self.__dict__.items():
            formatted_value = value
            if isinstance(value, list):
                formatted_value = ""
                for item in value:
                    formatted_value += str(item) + " "
                formatted_value = "\"" + formatted_value.strip() + "\""

            elif isinstance(value, str):
                try:
                    formatted_value = int(value)
                except:
                    formatted_value = "\"" + value.strip() + "\""
                
            map_["columns"] += attr_name
            map_["values"] += str(formatted_value)
            it+=1
        
        print(len(self.__dict__.items()))
        

        map_["columns"] = map_["columns"].replace(map_["columns"].split(", ")[-1], "") + ") "
        map_["values"] = map_["values"].replace(map_["values"].split(", ")[-1], "") + ") "

        query += map_["columns"] + map_["values"]
        print(query)



        # cur.execute('Select * from test')

        # rows = cur.fetchall()

        # for row in rows:
        #     print(row)
        

    def describe(self):
        for (attr_name, value) in self.__dict__.items():
            print(attr_name, "=>", value)

In [161]:
columns = []
for (month, item) in data.items():
    for (day_of_month, item) in item.items():
        for (key, value) in item.items():
            columns.append(key)

columns = set(columns)




for (month, item) in data.items():
    for (day_of_month, item) in item.items():
        mana = Mana(columns)
        mana.__setattr__("month", month)
        mana.__setattr__("day", day_of_month)
        for (key, value) in item.items():
            mana.__setattr__(key, value)
        mana.__setattr__("id", generate_mana_id(month, day_of_month, item["lohateny"], item["vakiteny"]))
        mana.describe()
        # mana.insert_into_db("baiboly copy.db")
        break
    break



teny fampanantenana => 
hosaintsaino => 
Lesona => 
Fanomezan-toky => 
vavaka => 
teny fampaherezana => 
fampieritreretana => 
Fanontaniana => 
Vavaka => 
Toetran’ Andriamanitra ny mahatadidy izay ampanantenainy, fa isika koa ve mahatadidy mandrakariva ny maha-saro-bidy ny fahatsiarovana Azy? => 
famapaherezana => 
Miambena => 
Toro-hevitra => 
fampidirana ny Epistily voalohan'i Petera => 
lohanteny => 
tanjona => 
Fampanantenana => 
Fampaherezana => 
FAMPIDIRANA GENESISY => 
Soratra Masina ho an’ny herinandro  => 
Famporisihina => 
lohateny => VEHIVAVY NITOLAGAGA
Fiderana => 
Toetra mila ananana => 
Hosaintsainina => 
Saintsaino => 
Inona no tantaranao? => 
Hafatra mahasoa => 
Saintsainina => 
Inona no baiko? => 
antso => 
vakiteny => LIOKA 11. 27-36
content => ['Mihaino ny teniny: Rehefa tena mihaino amin’ny fo tsotra ny Tenin’i Jesoa sy ny Tenin’Andriamanitra dia hahatsapa ny hasarobidiny tahaka ity vehivavy ity, hany ka nitolagaga sy loa-bolana hoe: “Sambatra ny kibo izay nitondra 